# Установка зависимостей

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
from IPython.utils import io
import torch
import time
from torch import optim
from torch.nn import functional as F
import shutil
import pickle
import os
import re

with io.capture_output() as captured:
  !pip install transformers sentencepiece sentence_transformers xmltodict pypinyin pykakasi

from sentence_transformers import SentenceTransformer, SentencesDataset, LoggingHandler, losses
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import SentenceEvaluator
from torch.utils.data import DataLoader
from datetime import datetime
import math

from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm.notebook import tqdm

with io.capture_output() as captured:
  !pip install lingtrain_aligner==1.0.2 dateparser razdel

from razdel import sentenize
from lingtrain_aligner import preprocessor, splitter, aligner, resolver, reader, helper, vis_helper, metrics
from lingtrain_aligner.saver import save_plain_text
from scipy.spatial.distance import cosine

sns.set()

# Вспомогательные функции

In [ ]:
import razdel

def split_sentences(text):
    return [sentence.text for sentence in razdel.sentenize(text)]

In [ ]:
def prepare_text(text):
    text = re.sub(r'([!?\.:,])(?!\.)', r'\1 ', text)
    # text = re.sub(':\n- ', '. ', text)
    text = re.sub('\s+', ' ', text)
    return text.strip()

In [ ]:
def clean_up(sents):
    map = {
        'ь:': 'ь.', 'Ē~ӽ': 'Ех', 'ѵ': 'у', 'Ē': 'Е', 'ā': 'а', 'ñ': '', '×': 'у',
        'ī': 'и', 'Ō': 'О', 'ō': 'о', 'р̃': 'р̌', 'р̆': 'р̌', '~': '-',
        '§.': '', 'ÿ': 'у', 'ʻ': '', 'ʼ': "'", 'р˘': 'р̌', 'р̀': 'р̌',
        '́': '', '̃': '', '̄': '', '̉': '', '̍': '', '̔': 'р', '̣': '',
        '̥': '', '̨': '', '̬': '', '̓': '', 'џ': 'ц', 'ґ': 'г', 'Ҥ': 'Ӈ',
        'Ң': 'Ӈ', 'ң': 'ӈ', 'Ӊ': 'Ӈ', 'ӊ': 'ӈ', 'ҧ': 'п', 'Ұ': 'ӻ',
        'ӗ': 'е', 'Өа х': 'Х', 'ӯ': 'у', 'ӱ': 'у', 'ӧ': 'ё', 'ҳ': 'х',
        '՚': "'", 'ḗ': 'е', 'ṓ': 'о', 'ṕ': 'р̌', '‑': '-', '‐': '-',
        '–': '-', '—': '-', '‘': "'", '’': "'", '“': '"', '”': '"',
        '«': '"', '»': '"', '•': '', '…': '...', '▸': '', 'Р̆': 'Р̌',
        'ē': 'е', 'Ӿ ': '! ', 'Ӿ" ': '!" ', "\\'": "'"
    }
    cleaned_sents = []
    for sent in sents:
        for old, new in map.items():
            sent = sent.replace(old, new)
        cleaned_sents.append(sent.replace("\\", ""))
    return cleaned_sents

In [ ]:
def find_file(files, pattern):
    for f in files:
        if re.fullmatch(pattern, f):
            return f
    return None

In [ ]:
#@title Главная функция

def align_texts(path_n, path_r, output_dir):
    with open(path_n, encoding='utf-8') as f:
        text1 = prepare_text(f.read())
    with open(path_r, encoding='utf-8') as f:
        text2 = prepare_text(f.read())

    # разбиваем на предложения, дополнительно чистим
    sents1 = clean_up(split_sentences(text1))
    sents2 = clean_up(split_sentences(text2))

    # задаём путь к временному индексу
    db_path = os.path.join(output_dir, 'alignment.db')
    if os.path.exists(db_path):
        os.unlink(db_path)  # удаляем, если остался с предыдущего запуска

    # записываем предложения в индекс для выравнивания
    aligner.fill_db(db_path, LANG_FROM, LANG_TO, sents1, sents2)

    # вычисляем динамический размер окна
    num_sents1 = len(sents1)
    num_sents2 = len(sents2)
    avg_len = (num_sents1 + num_sents2) / 2
    window = int(min(50, max(10, avg_len * 0.1)))

    # выравниваем с учётом окна и других параметров
    aligner.align_db(db_path, model_name=MODEL_NAME, model=MODEL,
                    batch_size=200, window=window, embed_batch_size=5,
                    normalize_embeddings=True, show_progress_bar=False,
                    shift=0)

    # несколько проходов разрешения конфликтов
    for step in range(3):
        conflicts, rest = resolver.get_all_conflicts(
            db_path,
            min_chain_length=2 + step,           # увеличиваем допустимую цепочку
            max_conflicts_len=6 * (step + 1),    # увеличиваем размер конфликтного блока
            handle_start=True,
            handle_finish=True
        )
        resolver.resolve_all_conflicts(db_path, conflicts, MODEL_NAME, model=MODEL)
        if len(rest) == 0:
            break  # если нечего решать — выходим раньше

    # финальный проход разрешения конфликтов, на всякий
    conflicts, _ = resolver.get_all_conflicts(
        db_path,
        min_chain_length=2,
        max_conflicts_len=26,
        handle_start=True,
        handle_finish=True
    )
    resolver.resolve_all_conflicts(db_path, conflicts, MODEL_NAME, model=MODEL)

    # визуализируем выравнивание для наглядной проверки
    vis_helper.visualize_alignment_by_db(
        db_path,
        output_path=os.path.join(output_dir, 'alignment_vis.png'),
        batch_size=800,
        size=(900, 900),
        lang_name_from=LANG_FROM,
        lang_name_to=LANG_TO,
        plt_show=False
    )

    # достаём финальные пары строк
    lines_from, lines_to = helper.read_processing(db_path)

    # фильтрация по относительной длине
    MAX_LEN_RATIO = 1.5

    length_ids = []
    for i, (s1, s2) in enumerate(zip(lines_from, lines_to)):
        len1 = len(s1.strip().split())
        len2 = len(s2.strip().split())
        if len1 == 0 or len2 == 0:
            continue
        ratio = max(len1 / len2, len2 / len1)
        if ratio <= MAX_LEN_RATIO:
            length_ids.append(i)

    # оставляем только пары с допустимым соотношением длины
    lines_from = np.array(lines_from)[length_ids]
    lines_to = np.array(lines_to)[length_ids]

    # считаем их косинусное сходство
    vec_from = aligner.get_line_vectors(lines_from, model_name=MODEL_NAME, model=MODEL)
    vec_to = aligner.get_line_vectors(lines_to, model_name=MODEL_NAME, model=MODEL)

    # считаем сходство между векторами, отбрасываем те, что ниже порога
    sims = [1 - cosine(x, y) if np.any(x) and np.any(y) else 0 for x, y in zip(vec_from, vec_to)]
    ids = [i for i, sim in enumerate(sims) if sim >= SENT_SIM_THRESHOLD]

    # фильтруем только те пары, которые прошли по порогу
    lines_from_filtered = np.array(lines_from)[ids]
    lines_to_filtered = np.array(lines_to)[ids]

    # убираем повторно используемые переводы (один ко многим)
    used_targets = set()
    filtered_pairs = []
    for s, t in zip(lines_from_filtered, lines_to_filtered):
        if t not in used_targets:
            filtered_pairs.append((s, t))
            used_targets.add(t)
    # перезапись
    lines_from_filtered, lines_to_filtered = zip(*filtered_pairs)

    print(f'N: {len(sents1)} --> {len(lines_from_filtered)}')
    print(f'R: {len(sents2)} --> {len(lines_to_filtered)}')

    # вычисляем chain_score
    score = metrics.chain_score(db_path)
    print(f'Chain score: {score:.4f}')

    # задаём порог
    CHAIN_SCORE_THRESHOLD = 0.7

    if score < CHAIN_SCORE_THRESHOLD:
        print(f'Skipping saving due to low chain_score: {score:.4f}')
        return 0, len(sents1), len(sents2), ''

    # сохраняем в нужном нам формате
    out_path = os.path.join(output_dir, 'n_r_p.txt')
    with open(out_path, 'w', encoding='utf-8') as f:
        for i, (s, t) in enumerate(zip(lines_from_filtered, lines_to_filtered), 1):
            f.write(f'{i}> {s}\n{i}= {t}\n\n')

    return len(lines_from_filtered), len(sents1), len(sents2), out_path

# Основной цикл

In [ ]:
!gdown --folder 1XMOKBmIwSDQkrn3WHmLZExRwOzOsGcQC

Retrieving folder contents
Processing file 1MIidLg2ZkBPeMbwD7tm--Q5x0fwHt_hO config.json
Processing file 1RWx7ZnTDUDJgLn7d5q7EKslyIUst4YnM model.safetensors
Processing file 1ZKeCVKDWSx0mNCa0eKO3JiStkVUbiQP2 special_tokens_map.json
Processing file 1V5Q1Wil0tWumsBIMakgFNasQLv_nHWUp tokenizer_config.json
Processing file 12X_PYHnvMLOhfmbbhnKU-BjFskMWDvku tokenizer.json
Processing file 1kEfQ8sxO3vHajrMbVSu615PSu7VRziE- vocab.txt
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1MIidLg2ZkBPeMbwD7tm--Q5x0fwHt_hO
To: /content/nivkh_v1/config.json
100% 818/818 [00:00<00:00, 4.32MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1RWx7ZnTDUDJgLn7d5q7EKslyIUst4YnM
From (redirected): https://drive.google.com/uc?id=1RWx7ZnTDUDJgLn7d5q7EKslyIUst4YnM&confirm=t&uuid=7f832acc-bcd0-45ef-9939-879bdca1fbe8
To: /content/nivkh_v1/model.safetensors
100% 541M/541M [00:06<00:00, 78.8MB/s

In [ ]:
from sentence_transformers import SentenceTransformer

model_path = '/content/nivkh_v1'
model = SentenceTransformer(model_path)

In [ ]:
LANG_FROM = 'nivkh'
LANG_TO = 'ru'
MODEL_NAME = '/content/nivkh_v1'
SENT_SIM_THRESHOLD = 0.7
MODEL = model
ROOT_FOLDER = '/content/drive/MyDrive/FilteredNivkhTxt'

In [ ]:
total_raw_n = 0
total_raw_r = 0
total_filtered = 0
valid_files = []

for root, dirs, files in tqdm(list(os.walk(ROOT_FOLDER))):
    src_file = find_file(files, r'(n(_am)?\.txt)')
    tgt_file = find_file(files, r'r\.txt')
    if src_file and tgt_file:
        path_n = os.path.join(root, src_file)
        path_r = os.path.join(root, tgt_file)
        print(path_n, path_r)
        try:
            filtered, raw_n, raw_r, out_path = align_texts(path_n, path_r, root)
            total_filtered += filtered
            total_raw_n += raw_n
            total_raw_r += raw_r
            valid_files.append(out_path)
        except Exception as e:
            print(f'Error in {root}: {e}')
    print('\n', '-'*1000, '\n')

print(len(valid_files)) # 253/331 (338 -- 7 с ошибкой)
print(f'N.0: {total_raw_n}') # 7956 -- изначально нивхских предложений
print(f'R.0: {total_raw_r}') # 7700 -- изначально русских предложений
print(f'--> N_R: {total_filtered}') # 2067 -- получившиеся пары
print(f'N.f: {total_raw_n - total_filtered} ({(total_raw_n - total_filtered)/total_raw_n:.1%})') # 5889 (74.0%) -- отфильтровано нивхских
print(f'R.f: {total_raw_r - total_filtered} ({(total_raw_r - total_filtered)/total_raw_r:.1%})') # 5633 (73.2%) -- отфильтровано русских

  0%|          | 0/338 [00:00<?, ?it/s]


 --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Aligning without segments.
tasks amount: 1
batch: 0 (0)
conflicts to solve: 2
total conflicts: 2



100%|██████████| 2/2 [00:00<00:00,  6.32it/s]


conflicts to solve: 0
total conflicts: 0


...

Aligning without segments.
tasks amount: 1
batch: 0 (0)
conflicts to solve: 2
total conflicts: 2



100%|██████████| 2/2 [00:00<00:00, 12.07it/s]


conflicts to solve: 0
total conflicts: 0



0it [00:00, ?it/s]


N: 9 --> 2
R: 8 --> 2
Chain score: 1.0000

 --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import json

valid_files = [f for f in valid_files if f != '']
print(len(valid_files))
with open('/content/drive/MyDrive/FilteredNivkhTxt/valid.json', 'w') as f:
    json.dump(valid_files, f)

253


In [ ]:
from tqdm.auto import tqdm
import json
import shutil
import os

with open('/content/drive/MyDrive/FilteredNivkhTxt/valid.json', 'r') as f:
    valid_files = json.load(f)
target_dir = '/content/drive/MyDrive/FilteredNivkhTxt/valid_files'
os.makedirs(target_dir, exist_ok=True)
for file_path in tqdm(valid_files):
    src_subfolder = os.path.basename(os.path.dirname(file_path))
    new_name = f'{src_subfolder}.txt'
    new_path = os.path.join(target_dir, new_name)
    shutil.copy(file_path, new_path)

  0%|          | 0/253 [00:00<?, ?it/s]